In [1]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [2]:
# Connect to the MCP server using the editing preset
server_params = StdioServerParameters(
    command='python',
    args=['-m', 'mcp_this', '--preset', 'editing'],
)

async with stdio_client(server_params) as (read, write):  # noqa: SIM117
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print("Available editing tools:\n-----------------------\n")
        for tool in tools.tools:
            print(f"- `{tool.name}`: {tool.description}")

        # Test create-file tool
        print("\n\nTesting create-file tool...")
        create_result = await session.call_tool(
            'create-file',
            {
                'path': 'temp/example.txt',
                'content': 'Hello from MCP editing tools!\nThis is a test file.',
            },
        )
        print(create_result.content[0].text)

        # Test edit-file tool
        print("\n\nTesting edit-file tool...")
        edit_result = await session.call_tool(
            'edit-file',
            {
                'file': 'temp/example.txt',
                'operation': 'insert_after',
                'anchor': 'Hello from MCP editing tools!',
                'content': 'This line was added using edit-file!',
            },
        )
        print(edit_result.content[0].text)

        # Test create-directory tool
        print("\n\nTesting create-directory tool...")
        mkdir_result = await session.call_tool(
            'create-directory',
            {
                'path': 'temp/nested/example/directory',
            },
        )
        print(mkdir_result.content[0].text)

Available editing tools:
-----------------------

- `create-file`: TOOL DESCRIPTION:

Create a new file with specified content

Examples:
  - create_file(path="/path/to/new_file.py", content="def main():\n    print('Hello world')\n\nif __name__ == '__main__':\n    main()")  # create a Python script
  - create_file(path="/path/to/config.json", content="{\n  \"name\": \"project\",\n  \"version\": \"1.0.0\"\n}")  # create a JSON config file
  - create_file(path="/path/to/.gitignore", content="node_modules/\n*.log\n.DS_Store")  # create a gitignore file
  - create_file(path="/path/to/empty_file.txt")  # create an empty file

COMMAND CALLED:

`if [ -f "<<path>>" ]; then
  echo "Error: File '<<path>>' already exists. Use edit-file tool to modify existing files.";
  exit 1;
else
  mkdir -p "$(dirname "<<path>>")" &&
  if [ -n "<<content>>" ]; then
    printf '%s' "<<content>>" > "<<path>>";
  else
    touch "<<path>>";
  fi &&
  echo "File created successfully: <<path>>" &&
  ls -la "<<path>>

## MCP-This Editing Tools Example

This notebook demonstrates how to use the MCP-This server with the built-in `editing` preset configuration.

### Key Features:

1. **Built-in Preset**: Uses `--preset editing` to load the editing toolset without needing a separate config file
2. **File Creation**: Create new files with specified content
3. **File Editing**: Modify existing files with various operations (insert, replace, delete)
4. **Directory Creation**: Create directories and nested directory structures

### Available Operations for edit-file:

- `insert_after`: Insert content after a matching line
- `insert_before`: Insert content before a matching line
- `replace`: Replace a line that matches a pattern
- `delete`: Delete lines that match a pattern
- `replace_range`: Replace a range of line numbers

### Usage in Claude Desktop:

To use these editing tools in Claude Desktop, configure your MCP server with:

```json
{
  "mcpServers": {
    "mcp-this-editing": {
      "command": "uvx",
      "args": ["mcp-this", "--preset", "editing"]
    }
  }
}
```